In [25]:
import pandas as pd

# Load the CSV file
file_path = '/Users/lebakuprathyushkumarreddy/Downloads/pavement_with_crashes_for_each_collisiontype_csvfile/merged_csv_file_using_geopandas.csv'
df = pd.read_csv(file_path)

data = df[df["CRCOMNNR"] == 1]
# Grouping the data by 'OBJECTID' and counting the crashes
crash_counts = data.groupby('OBJECTID').size()
# Creating a new dataframe with OBJECTID and the crash counts
pavement_section_data = data.drop_duplicates(subset='OBJECTID').set_index('OBJECTID')
pavement_section_data['Crash_Count'] = crash_counts

# Resetting the index to include OBJECTID as a column
pavement_section_data.reset_index(inplace=True)

# The final dataframe is stored in pavement_section_data

pavement_section_data["Crash_Rate"]= pavement_section_data["Crash_Count"] *100000000/(pavement_section_data["AADT"]*pavement_section_data["PMIS_LENGTH"]*365)

print(f"Total crashes is {pavement_section_data['Crash_Count'].sum()} ")



Total crashes is 6084 


/var/folders/yl/h3jt_tns2s14c8y5w9qptb6c0000gn/T/ipykernel_30909/1318962289.py:5: DtypeWarning: Columns (143,172,176,177,178,179,188,203,204,207,208,209,210,211,212,243,244,252,257,258,267,268,274,275,281) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [26]:
iri_bins = [0, 95, 170, float('inf')]
iri_labels = ['Good', 'Fair', 'Poor']
pavement_section_data['IRI_Category'] = pd.cut(pd.to_numeric(pavement_section_data['IRI'], errors='coerce'), bins=iri_bins, labels=iri_labels)
mean_crash_rates = pavement_section_data.groupby('IRI_Category')['Crash_Rate'].mean() # to find mean crash rate in each segment
print("Mean crash rates per million vmt:\n", mean_crash_rates)

Mean crash rates per million vmt:
 IRI_Category
Good     56.355456
Fair     72.519177
Poor    119.595094
Name: Crash_Rate, dtype: float64


In [27]:
import pandas as pd
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Assuming pavement_section_data is your DataFrame
# Check and handle missing values
pavement_section_data.dropna(subset=['Crash_Rate', 'IRI_Category'], inplace=True)

# Convert IRI_Category to categorical if not already
pavement_section_data['IRI_Category'] = pd.Categorical(pavement_section_data['IRI_Category'])

# Perform the Tukey's test
tukey_results = pairwise_tukeyhsd(endog=pavement_section_data['Crash_Rate'], 
                                  groups=pavement_section_data['IRI_Category'], 
                                  alpha=0.05)

# Print summary
print(tukey_results.summary())


 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
  Fair   Good -16.1637 0.0004 -26.0999 -6.2275   True
  Fair   Poor  47.0759    0.0  28.2187 65.9331   True
  Good   Poor  63.2396    0.0  44.8511 81.6281   True
-----------------------------------------------------


In [28]:
#Fishers LSD Test 
import pandas as pd
import numpy as np
from scipy.stats import f, t

# Perform ANOVA to test for overall group differences
groups = [pavement_section_data[pavement_section_data['IRI_Category'] == label]['Crash_Rate'] for label in iri_labels]
n_groups = len(groups)
n_total = len(pavement_section_data['Crash_Rate'])
n_obs = [len(group) for group in groups]
mean_group = [group.mean() for group in groups]
overall_mean = pavement_section_data['Crash_Rate'].mean()

# Calculate the sum of squares between groups (SSB)
SSB = sum([n * (mean - overall_mean)**2 for n, mean in zip(n_obs, mean_group)])

# Calculate the mean square between groups (MSB)
MSB = SSB / (n_groups - 1)

# Calculate the mean square error (MSE)
SSE = sum([(n - 1) * group.var() for n, group in zip(n_obs, groups)])
MSE = SSE / (n_total - n_groups)

# Calculate the F-statistic
F = MSB / MSE

# Calculate the critical value from the F-distribution
alpha = 0.05
df1 = n_groups - 1
df2 = n_total - n_groups
critical_value = f.ppf(1 - alpha, df1, df2)

# Perform pairwise comparisons using Fisher's LSD and calculate p-values
from itertools import combinations
pairwise_comparisons = list(combinations(iri_labels, 2))
for label1, label2 in pairwise_comparisons:
    group1 = pavement_section_data[pavement_section_data['IRI_Category'] == label1]['Crash_Rate']
    group2 = pavement_section_data[pavement_section_data['IRI_Category'] == label2]['Crash_Rate']
    
    # Calculate the pooled standard error
    pooled_var = ((n_obs[0] - 1) * group1.var() + (n_obs[1] - 1) * group2.var()) / (n_obs[0] + n_obs[1] - 2)
    pooled_se = np.sqrt(pooled_var * (1/n_obs[0] + 1/n_obs[1]))
    
    # Calculate the LSD
    lsd = abs(mean_group[0] - mean_group[1]) / pooled_se
    # Calculate the critical LSD value
    critical_lsd = t.ppf(1 - alpha/2, df2) * pooled_se
   
    # Calculate the p-value
    p_value = 2 * (1 - t.cdf(abs(lsd), df2))
    
    if lsd > critical_lsd:
        print(f"Comparison between '{label1}' and '{label2}': LSD = {lsd:.4f} (significant), p-value = {p_value:.4f}")
        print(f"Value of LSD: {lsd}")
        print(f"Value of Crirical LSD: {critical_lsd}")

    else:
        print(f"Comparison between '{label1}' and '{label2}': LSD = {lsd:.4f} (not significant), p-value = {p_value:.4f}")
        print(f"Value of LSD: {lsd}")
        print(f"Value of Crirical LSD: {critical_lsd}")


Comparison between 'Good' and 'Fair': LSD = 4.5488 (not significant), p-value = 0.0000
Value of LSD: 4.548827948480511
Value of Crirical LSD: 6.968307337928697
Comparison between 'Good' and 'Poor': LSD = 2.5514 (not significant), p-value = 0.0108
Value of LSD: 2.551442539311478
Value of Crirical LSD: 12.423415649770298
Comparison between 'Fair' and 'Poor': LSD = 2.4048 (not significant), p-value = 0.0163
Value of LSD: 2.4047625373288244
Value of Crirical LSD: 13.181189693507598
